In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

import time
import datetime as dt
import toad

In [2]:
import warnings
warnings.filterwarnings("ignore")

# One-click Model

In [120]:
start_model = time.time()

## Shared Funtion

In [121]:
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(include='object').columns.tolist()
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## I. Application

### 1.Clean Data

#### Missing Values

In [122]:
def prep_fill_missing_app(dataframe):
    # convert days to years
    def days_to_year(dataframe):
        for col in dataframe.columns[dataframe.columns.str.startswith('DAYS')]:
            dataframe[f'YEARS{col[4:]}'] = np.ceil(dataframe[col] / -365)
        return dataframe
    dataframe = days_to_year(dataframe)
    # delete invalid in years (< 0)
    dataframe[['YEARS_BIRTH','YEARS_EMPLOYED','YEARS_REGISTRATION','YEARS_ID_PUBLISH', 'YEARS_LAST_PHONE_CHANGE']] = dataframe[['YEARS_BIRTH','YEARS_EMPLOYED','YEARS_REGISTRATION','YEARS_ID_PUBLISH', 'YEARS_LAST_PHONE_CHANGE']].applymap(lambda x: np.nan if x < 0 else x)
    # delete invalid in days (> 0)
    dataframe[['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE']] = dataframe[['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE']].applymap(lambda x: np.nan if x > 0 else x)

    # delete invalid in gender
    dataframe['CODE_GENDER'] = np.where(dataframe['CODE_GENDER'] == 'XNA', np.nan, dataframe['CODE_GENDER'])


    # drop useless columns
    avg_col = list(dataframe.columns[dataframe.columns.str.endswith('AVG')])
    mode_col = list(dataframe.columns[dataframe.columns.str.endswith('MODE')])
    med_col = list(dataframe.columns[dataframe.columns.str.endswith('MEDI')])
    addition_drop = []
    col_to_drop = avg_col + mode_col + med_col + addition_drop

    # return col_to_drop
    # dataframe.drop(columns=col_to_drop, inplace=True)
    return dataframe

# def lazy_fill_missing_app(dataframe_train, dataframe_test):
#     col_num = dataframe_train.select_dtypes('number').columns.drop('TARGET')
#     for col in col_num:
#         median_value = dataframe_train[col].median()
#         dataframe_train[col] = dataframe_train[col].fillna(median_value)
#         dataframe_test[col] = dataframe_test[col].fillna(median_value)
#     col_object = dataframe_train.select_dtypes('object').columns
#     for col in col_object:
#         mode_value = dataframe_train[col].mode()[0]
#         dataframe_train[col] = dataframe_train[col].fillna(mode_value)
#         dataframe_test[col] = dataframe_test[col].fillna(mode_value)
#     # dataframe_train['AGE_BIN'] = pd.cut(dataframe_train['YEARS_BIRTH'], 10)
#     # # dataframe_test['AGE_BIN'] = pd.cut(dataframe_test['YEARS_BIRTH'], 10)
#     # missing_col = (dataframe_train.isna().sum()[dataframe_train.isna().sum() > 0].index).to_list()
#     # col_to_group = 'AGE_BIN'
#     # col_to_impute = missing_col
#     # # vòng for dùng để impute cho nhiều cột
#     # for i, col in enumerate(col_to_impute):
#     #     # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
#     #     grouped = dataframe_train.groupby([col_to_group])[col]
#     #     for val_to_group, group_data in grouped:
#     #         if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
#     #             mean_value = group_data.mean()
#     #             # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
#     #             dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
#     #             dataframe_test.loc[(dataframe_test['YEARS_BIRTH'].between(val_to_group[0].left, val_to_group[0].right, inclusive='right')) & (dataframe_test[col].isna()), col] = mean_value
#     #         else:  # fill biến câte = mode của từng group_data
#     #             mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
#     #             dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
#     #             dataframe_test.loc[(dataframe_test['YEARS_BIRTH'].between(val_to_group[0].left, val_to_group[0].right, inclusive='right')) & (dataframe_test[col].isna()), col] = mode_value
#     # dataframe_train.drop(columns='AGE_BIN', inplace=True)
#     return dataframe_train, dataframe_test
    

def fill_missing_app(dataframe_train, dataframe_test):
    # 1
    mode_gender = dataframe_train['CODE_GENDER'].mode()
    dataframe_train['CODE_GENDER'].fillna(mode_gender, inplace=True)
    dataframe_test['CODE_GENDER'].fillna(mode_gender, inplace=True)
    # 2
    mode_year_last_phone = dataframe_train['YEARS_LAST_PHONE_CHANGE'].mode()
    dataframe_train['YEARS_LAST_PHONE_CHANGE'].fillna(mode_year_last_phone, inplace=True)
    dataframe_test['YEARS_LAST_PHONE_CHANGE'].fillna(mode_year_last_phone, inplace=True)
    # 3
    mode_day_last_phone = dataframe_train['DAYS_LAST_PHONE_CHANGE'].mode()
    dataframe_train['DAYS_LAST_PHONE_CHANGE'].fillna(mode_day_last_phone, inplace=True)
    dataframe_test['DAYS_LAST_PHONE_CHANGE'].fillna(mode_day_last_phone, inplace=True)
    # 4
    mean_fam_mem = dataframe_train['CNT_FAM_MEMBERS'].mean()
    dataframe_train['CNT_FAM_MEMBERS'].fillna(mean_fam_mem, inplace=True)
    dataframe_test['CNT_FAM_MEMBERS'].fillna(mean_fam_mem, inplace=True)
    
    # 5, 6, 7, 8, 9
    dataframe_train['AGE_BIN'] = pd.cut(dataframe_train['YEARS_BIRTH'], 10)
    dataframe_test['AGE_BIN'] = pd.cut(dataframe_test['YEARS_BIRTH'], 10)

    col_to_group = 'AGE_BIN'
    col_to_impute = ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        grouped = dataframe_train.groupby([col_to_group])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mode_value
    
    # 10, 11, 12, 13
    col_to_group = 'NAME_INCOME_TYPE'
    col_to_impute = ['OWN_CAR_AGE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        grouped = dataframe_train.groupby([col_to_group])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mode_value
    
    # 14
    col_to_group = 'REGION_RATING_CLIENT'
    col_to_impute = ['DEF_30_CNT_SOCIAL_CIRCLE']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        grouped = dataframe_train.groupby([col_to_group])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mode_value
    
    # 15, 16, 17, 18, 19, 20
    col_to_group = 'AMT_GOODS_PRICE'
    col_to_impute = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        dataframe_train[f'{col_to_group}_BIN'] = pd.cut(dataframe_train[col_to_group], 10)
        grouped = dataframe_train.groupby([f'{col_to_group}_BIN'])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[f'{col_to_group}_BIN'] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group].between(val_to_group[0].left, val_to_group[0].right, inclusive='right')) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[f'{col_to_group}_BIN'] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group].between(val_to_group[0].left, val_to_group[0].right, inclusive='right')) & (dataframe_test[col].isna()), col] = mode_value
    dataframe_train.drop(columns=f'{col_to_group}_BIN', inplace=True)
    
    # 21
    col_to_group = 'AMT_INCOME_TOTAL'
    col_to_impute = ['YEARS_EMPLOYED']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        dataframe_train[f'{col_to_group}_BIN'] = pd.cut(dataframe_train[col_to_group], 10)
        grouped = dataframe_train.groupby([f'{col_to_group}_BIN'])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[f'{col_to_group}_BIN'] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group].between(val_to_group[0].left, val_to_group[0].right, inclusive='right')) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[f'{col_to_group}_BIN'] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group].between(val_to_group[0].left, val_to_group[0].right, inclusive='right')) & (dataframe_test[col].isna()), col] = mode_value
    dataframe_train.drop(columns=f'{col_to_group}_BIN', inplace=True)
    
    dataframe_train.drop(columns='AGE_BIN', inplace=True)
    dataframe_test.drop(columns='AGE_BIN', inplace=True)
    
    return dataframe_train, dataframe_test

#### Outliers

In [123]:
def clipping_outliers_app(dataframe_train, dataframe_test):
    # for categorical variables which has density < 5%
    columns = dataframe_train.select_dtypes('object').columns
    for col in columns:
        if dataframe_train[col].nunique() > 2:
            cond = dataframe_train[col].value_counts(normalize=True)[dataframe_train[col].value_counts(normalize=True) < 0.05].index.to_list()
            dataframe_train.loc[dataframe_train[col].isin(cond), col] = 'Others'
            dataframe_test.loc[dataframe_test[col].isin(cond), col] = 'Others'
    # for numeric variables lied in 1% of distribution
    col_to_check = dataframe_train.select_dtypes('number').columns.drop(['SK_ID_CURR', 'TARGET'])
    for col in col_to_check:
        upper_limit = dataframe_train[col].quantile(.99)
        lower_limit = dataframe_train[col].quantile(.01)
        dataframe_train.loc[dataframe_train[col] >= upper_limit, col] = upper_limit
        dataframe_train.loc[dataframe_train[col] <= lower_limit, col] = lower_limit
        dataframe_test.loc[dataframe_test[col] >= upper_limit, col] = upper_limit
        dataframe_test.loc[dataframe_test[col] <= lower_limit, col] = lower_limit
    return dataframe_train, dataframe_test


### 2.Feature Engineering

In [ ]:
def create_feature_app(dataframe):
    # flag obs and def on 30, 60 days
    dataframe['30_def'] = np.where(dataframe['DEF_30_CNT_SOCIAL_CIRCLE'] == 0, 0, 1)
    dataframe['60_def'] = np.where(dataframe['DEF_60_CNT_SOCIAL_CIRCLE'] == 0, 0, 1)
    dataframe['30_OBS'] = np.where(dataframe['OBS_30_CNT_SOCIAL_CIRCLE'] == 0, 0, 1)
    dataframe['60_OBS'] = np.where(dataframe['OBS_60_CNT_SOCIAL_CIRCLE'] == 0, 0, 1)

    # Flag_document features - count and kurtosis
    docs = [f for f in dataframe.columns if 'FLAG_DOC' in f]
    dataframe['DOCUMENT_COUNT'] = dataframe[docs].sum(axis=1)
    dataframe['DOCUMENT_RATIO'] = dataframe['DOCUMENT_COUNT'] / len(docs)
    dataframe['NEW_DOC_KURT'] = dataframe[docs].kurtosis(axis=1)
    # "domain knowledge"
    dataframe['CREDIT_TERM'] = dataframe['AMT_ANNUITY'] / dataframe['AMT_CREDIT']
    dataframe['credit_downpayment'] = dataframe['AMT_GOODS_PRICE'] - dataframe['AMT_CREDIT']
    # Some simple new features (percentages)
    dataframe['DAYS_EMPLOYED_PERC'] = dataframe['DAYS_EMPLOYED'] / dataframe['DAYS_BIRTH']
    dataframe['INCOME_CREDIT_PERC'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['AMT_CREDIT']
    dataframe['INCOME_PER_PERSON'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['CNT_FAM_MEMBERS']
    dataframe['ANNUITY_INCOME_PERC'] = dataframe['AMT_ANNUITY'] / dataframe['AMT_INCOME_TOTAL']
    dataframe['PAYMENT_RATE'] = dataframe['AMT_ANNUITY'] / dataframe['AMT_CREDIT']
    dataframe['CREDIT_TO_GOODS_RATIO'] = dataframe['AMT_CREDIT'] / dataframe['AMT_GOODS_PRICE']
    dataframe['INCOME_TO_EMPLOYED_RATIO'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['DAYS_EMPLOYED']
    dataframe['INCOME_TO_BIRTH_RATIO'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['DAYS_BIRTH']
    dataframe['ID_TO_BIRTH_RATIO'] = dataframe['DAYS_ID_PUBLISH'] / dataframe['DAYS_BIRTH']
    dataframe['CAR_TO_BIRTH_RATIO'] = dataframe['OWN_CAR_AGE'] / dataframe['DAYS_BIRTH']
    dataframe['CAR_TO_EMPLOYED_RATIO'] = dataframe['OWN_CAR_AGE'] / dataframe['DAYS_EMPLOYED']
    dataframe['PHONE_TO_BIRTH_RATIO'] = dataframe['DAYS_LAST_PHONE_CHANGE'] / dataframe['DAYS_BIRTH']
    dataframe['APPS_GOODS_INCOME_RATIO'] = dataframe['AMT_GOODS_PRICE'] / dataframe['AMT_INCOME_TOTAL']
    dataframe['APPS_CNT_FAM_INCOME_RATIO'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['CNT_FAM_MEMBERS']
    dataframe['APPS_INCOME_EMPLOYED_RATIO'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['DAYS_EMPLOYED']


    # EXT_SOURCE_X FEATURE
    dataframe['APPS_EXT_SOURCE_MEAN'] = dataframe[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    dataframe['APPS_EXT_SOURCE_STD'] = dataframe[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    dataframe['APPS_EXT_SOURCE_STD'] = dataframe['APPS_EXT_SOURCE_STD'].fillna(dataframe['APPS_EXT_SOURCE_STD'].mean())
    dataframe['APP_SCORE1_TO_BIRTH_RATIO'] = dataframe['EXT_SOURCE_1'] / (dataframe['DAYS_BIRTH'] / 365.25)
    dataframe['APP_SCORE2_TO_BIRTH_RATIO'] = dataframe['EXT_SOURCE_2'] / (dataframe['DAYS_BIRTH'] / 365.25)
    dataframe['APP_SCORE3_TO_BIRTH_RATIO'] = dataframe['EXT_SOURCE_3'] / (dataframe['DAYS_BIRTH'] / 365.25)
    dataframe['APP_SCORE1_TO_EMPLOY_RATIO'] = dataframe['EXT_SOURCE_1'] / (dataframe['DAYS_EMPLOYED'] / 365.25)
    dataframe['APP_EXT_SOURCE_2*EXT_SOURCE_3*DAYS_BIRTH'] = dataframe['EXT_SOURCE_1'] * dataframe['EXT_SOURCE_2'] * dataframe['DAYS_BIRTH']
    dataframe['APP_SCORE1_TO_FAM_CNT_RATIO'] = dataframe['EXT_SOURCE_1'] / dataframe['CNT_FAM_MEMBERS']
    dataframe['APP_SCORE1_TO_GOODS_RATIO'] = dataframe['EXT_SOURCE_1'] / dataframe['AMT_GOODS_PRICE']
    dataframe['APP_SCORE1_TO_CREDIT_RATIO'] = dataframe['EXT_SOURCE_1'] / dataframe['AMT_CREDIT']
    dataframe['APP_SCORE1_TO_SCORE2_RATIO'] = dataframe['EXT_SOURCE_1'] / dataframe['EXT_SOURCE_2']
    dataframe['APP_SCORE1_TO_SCORE3_RATIO'] = dataframe['EXT_SOURCE_1'] / dataframe['EXT_SOURCE_3']
    dataframe['APP_SCORE2_TO_CREDIT_RATIO'] = dataframe['EXT_SOURCE_2'] / dataframe['AMT_CREDIT']
    dataframe['APP_SCORE2_TO_REGION_RATING_RATIO'] = dataframe['EXT_SOURCE_2'] / dataframe['REGION_RATING_CLIENT']
    dataframe['APP_SCORE2_TO_CITY_RATING_RATIO'] = dataframe['EXT_SOURCE_2'] / dataframe['REGION_RATING_CLIENT_W_CITY']
    dataframe['APP_SCORE2_TO_POP_RATIO'] = dataframe['EXT_SOURCE_2'] / dataframe['REGION_POPULATION_RELATIVE']
    dataframe['APP_SCORE2_TO_PHONE_CHANGE_RATIO'] = dataframe['EXT_SOURCE_2'] / dataframe['DAYS_LAST_PHONE_CHANGE']
    dataframe['APP_EXT_SOURCE_1*EXT_SOURCE_2'] = dataframe['EXT_SOURCE_1'] * dataframe['EXT_SOURCE_2']
    dataframe['APP_EXT_SOURCE_1*EXT_SOURCE_3'] = dataframe['EXT_SOURCE_1'] * dataframe['EXT_SOURCE_3']
    dataframe['APP_EXT_SOURCE_2*EXT_SOURCE_3'] = dataframe['EXT_SOURCE_2'] * dataframe['EXT_SOURCE_3']
    dataframe['APP_EXT_SOURCE_1*DAYS_EMPLOYED'] = dataframe['EXT_SOURCE_1'] * dataframe['DAYS_EMPLOYED']
    dataframe['APP_EXT_SOURCE_2*DAYS_EMPLOYED'] = dataframe['EXT_SOURCE_2'] * dataframe['DAYS_EMPLOYED']
    dataframe['APP_EXT_SOURCE_3*DAYS_EMPLOYED'] = dataframe['EXT_SOURCE_3'] * dataframe['DAYS_EMPLOYED']
    dataframe['EXT_SOURCES_PROD'] = dataframe['EXT_SOURCE_1'] * dataframe['EXT_SOURCE_2'] * dataframe['EXT_SOURCE_3']
    dataframe['EXT_SOURCES_WEIGHTED'] = dataframe.EXT_SOURCE_1 * 2 + dataframe.EXT_SOURCE_2 * 1 + dataframe.EXT_SOURCE_3 * 3
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        dataframe[feature_name] = eval('np.{}'.format(function_name))(
            dataframe[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)


    # other feature from better than 0.8
    dataframe['CREDIT_TO_GOODS_RATIO_2'] = dataframe['AMT_CREDIT'] / dataframe['AMT_GOODS_PRICE']
    dataframe['APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio'] = dataframe['AMT_INCOME_TOTAL'] / 12. - dataframe['AMT_ANNUITY']
    dataframe['APP_INCOME_TO_EMPLOYED_RATIO'] = dataframe['AMT_INCOME_TOTAL'] / dataframe['DAYS_EMPLOYED']
    dataframe['APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio'] = dataframe['DAYS_LAST_PHONE_CHANGE'] / dataframe['DAYS_EMPLOYED']
    dataframe['APP_DAYS_EMPLOYED_DAYS_BIRTH_diff'] = dataframe['DAYS_EMPLOYED'] - dataframe['DAYS_BIRTH']
    
    def get_age_label(days_birth):
        """ Return the age group label (int). """
        age_years = -days_birth / 365
        if age_years < 27: return 1
        elif age_years < 40: return 2
        elif age_years < 50: return 3
        elif age_years < 65: return 4
        elif age_years < 99: return 5
        else: return 0
    dataframe['AGE_RANGE'] = dataframe['DAYS_BIRTH'].apply(lambda x: get_age_label(x))
    dataframe['UNDER_40'] = np.where(dataframe['YEARS_BIRTH'] <= 40, 1, 0)
    dataframe['30_to_40'] = np.where(dataframe['YEARS_BIRTH'].between(30, 40), 1, 0)
    dataframe['FIRED_3YEARS'] = np.where(dataframe['YEARS_EMPLOYED'] <= 3, 1, 0)
    dataframe['FIRED_10YEARS'] = np.where(dataframe['YEARS_EMPLOYED'] <= 10, 1, 0)
    dataframe['LOYAL_EMPLOYEE'] = np.where(dataframe['YEARS_EMPLOYED'] > 10, 1, 0)
    dataframe['credit_to_annuity_ratio'] = dataframe['AMT_CREDIT'] / dataframe['AMT_ANNUITY']
    dataframe['payment_rate'] = dataframe['AMT_ANNUITY'] / dataframe['AMT_CREDIT']
    dataframe['HOUSETYPE_MODE_CHECK'] = np.where(dataframe['HOUSETYPE_MODE'] == 'block of flats', 1, 0)
    dataframe['NAME_TYPE_SUITE_CHECK'] = np.where(dataframe['NAME_TYPE_SUITE'] == 'Unaccompanied', 1, 0)
    dataframe['NAME_INCOME_TYPE_CHECK'] = np.where(dataframe['NAME_INCOME_TYPE'] == 'Working', 1, 0)
    dataframe['NAME_EDUCATION_TYPE_CHECK'] = np.where(dataframe['NAME_EDUCATION_TYPE'] == 'Secondary / secondary special', 1, 0)
    dataframe['NAME_FAMILY_STATUS_CHECK'] = np.where(dataframe['NAME_FAMILY_STATUS'] == 'Married', 1, 0)
    dataframe['NAME_HOUSING_TYPE_CHECK'] = np.where(dataframe['NAME_HOUSING_TYPE'] == 'House / apartment', 1, 0)
    dataframe['OCCUPATION_TYPE_CHECK'] = np.where(dataframe['OCCUPATION_TYPE'] == 'Married', 1, 0)
    dataframe['NAME_FAMILY_STATUS_CHECK'] = np.where(dataframe['NAME_FAMILY_STATUS'] == 'Laborers', 1, 0)
    dataframe['FONDKAPREMONT_MODE_CHECK'] = np.where(dataframe['FONDKAPREMONT_MODE'].isin(['reg oper account']), 1, 0)
    dataframe['WALLSMATERIAL_MODE_CHECK'] = np.where(dataframe['WALLSMATERIAL_MODE'].isin(['Panel']), 1, 0)
    dataframe['EMERGENCYSTATE_MODE_CHECK'] = np.where(dataframe['EMERGENCYSTATE_MODE'].isin(['No']), 1, 0)
    dataframe['CNT_CHILDREN_CHECK'] = np.where(dataframe['CNT_CHILDREN'] == 0, 1, 0)
    dataframe['CNT_FAM_MEMBERS_CHECK'] = np.where(dataframe['CNT_FAM_MEMBERS'] <= 2, 1, 0)
    dataframe['CNT_FAM_MEMBERS_CHECK'] = np.where(dataframe['CNT_FAM_MEMBERS'] <= 2, 1, 0)

    
    
    
    return dataframe

### Feature Selection

In [125]:
def select_feature(dataframe, target='TARGET', exclude_list=['SK_ID_CURR', 'TARGET'],
                   empty=0.5, iv=0.02, corr=0.8):
    train_selected, drop_lst= toad.selection.select(frame = dataframe,
                                                    target=dataframe[target], 
                                                    empty = empty, 
                                                    iv = iv, corr = corr, 
                                                    return_drop=True, 
                                                    exclude=exclude_list)
    # frame: dataset
    # target: target column
    # empty: remove feature with > 60% empty
    # iv: retain feature with iv > 0.02
    # corr: retain feature with corr < 0.9
    # return_drop = True: return the column dropped
    # exclude: a list of column that should not be assess and drop (id, label)
    print("keep:",train_selected.shape[1],
        "\ndrop iv:",len(drop_lst['iv']),
        "\ndrop empty:",len(drop_lst['empty']),
        "\ndrop corr:",len(drop_lst['corr']))
    return train_selected

### 4.Main Function

In [126]:
def application():
    # import data
    train = pd.read_csv('data/application_train.csv')
    test = pd.read_csv('data/application_test.csv')

    # fill missing
    
    train = prep_fill_missing_app(train)
    test = prep_fill_missing_app(test)
    
    train, test = fill_missing_app(train, test)
    # clipping outliers
    
    train, test = clipping_outliers_app(train, test)
    # Combine train and test back after cleaning
    df = pd.concat([train, test], axis=0)
    df = create_feature_app(df)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category=True)
    df.drop(columns='Unnamed: 0', inplace=True)
    train = df[df['TARGET'].notna()]
    test = df[df['TARGET'].isna()]
    train = select_feature(train)
    df = pd.concat([train, test[train.columns]], axis=0)
    print('"Application_Train_Test" final shape:', df.shape)
    return df

## II. Bureau

### 1.Clean Data

In [127]:
def fill_month_bal_mean_bureau(x):
    if abs(x) >= 30:
        x = abs(x)//30
        mean_value = np.mean(np.floor(np.arange(0, x)/ 12))
        return mean_value
    return 0

def lazy_fill_missing_bureau(dataframe_train, dataframe_test):
    col_num = dataframe_train.select_dtypes('number').columns
    for col in col_num:
        median_value = dataframe_train[col].median()
        dataframe_train[col] = dataframe_train[col].fillna(median_value)
        dataframe_test[col] = dataframe_test[col].fillna(median_value)
    col_object = dataframe_train.select_dtypes('object').columns
    for col in col_object:
        mode_value = dataframe_train[col].mode()[0]
        dataframe_train[col] = dataframe_train[col].fillna(mode_value)
        dataframe_test[col] = dataframe_test[col].fillna(mode_value)
    return dataframe_train, dataframe_test

def fill_missing_bureau(df_train, df_test, aggregated_bureau_balance):
    common_cols = df_train.columns.intersection(aggregated_bureau_balance.columns)
    common_cols = common_cols.drop(['MONTHS_BALANCE_MEAN', 'MONTHS_BALANCE_MAX'])

    df_train['MONTHS_BALANCE_MEAN'] = df_train['MONTHS_BALANCE_MEAN'].fillna(df_train['DAYS_CREDIT'].apply(lambda x: fill_month_bal_mean_bureau(x) if pd.notna(x) else None))
    df_test['MONTHS_BALANCE_MEAN'] = df_test['MONTHS_BALANCE_MEAN'].fillna(df_test['DAYS_CREDIT'].apply(lambda x: fill_month_bal_mean_bureau(x) if pd.notna(x) else None))
    
    df_train['MONTHS_BALANCE_MAX'] = df_train['MONTHS_BALANCE_MAX'].fillna(df_train['DAYS_CREDIT'].apply(lambda x: abs(x)//365 if pd.notna(x) else None))
    df_test['MONTHS_BALANCE_MAX'] = df_test['MONTHS_BALANCE_MAX'].fillna(df_test['DAYS_CREDIT'].apply(lambda x: abs(x)//365 if pd.notna(x) else None))

    #coi nhu cac khoan vay o trang thai unknown la X => de bang 1
    group = df_train[(df_train['STATUS_MEAN']==1)&(df_train['STATUS_MAX']==1)&(df_train['STATUS_FIRST']==1)]

    numeric_cols = group.select_dtypes(include=['number']).columns
    mean_values = group.groupby('MONTHS_BALANCE_MAX')[numeric_cols].mean()
    
    for col in common_cols:
        df_train[col] = df_train.apply(lambda row: mean_values.loc[row['MONTHS_BALANCE_MAX'], col] if pd.isna(row[col]) else row[col],
            axis=1)
        df_test[col] = df_test.apply(lambda row: mean_values.loc[row['MONTHS_BALANCE_MAX'], col] if pd.isna(row[col]) else row[col],
            axis=1)
        
    # fill nhung khoan vay da close coi nhu la tra dung han
    df_train['DAYS_CREDIT_ENDDATE'] = df_train['DAYS_CREDIT_ENDDATE'].fillna(df_train['DAYS_ENDDATE_FACT'])
    df_test['DAYS_CREDIT_ENDDATE'] = df_test['DAYS_CREDIT_ENDDATE'].fillna(df_test['DAYS_ENDDATE_FACT'])

    #fill nhung khoan vay active 
    df_train['DAYS_CREDIT_ENDDATE'] = df_train['DAYS_CREDIT_ENDDATE'].fillna(np.round(df_train[df_train['CREDIT_ACTIVE']=='Active']['DAYS_CREDIT_ENDDATE'].mean()))
    df_test['DAYS_CREDIT_ENDDATE'] = df_test['DAYS_CREDIT_ENDDATE'].fillna(np.round(df_train[df_train['CREDIT_ACTIVE']=='Active']['DAYS_CREDIT_ENDDATE'].mean()))

    # train_df['AMT_ANNUITY']
    # train_df['AMT_CREDIT_MAX_OVERDUE']
    # train_df['AMT_CREDIT_SUM_DEBT']
    # train_df['AMT_CREDIT_SUM_LIMIT']
    # train_df['AMT_CREDIT_SUM']
    df_train.drop(columns=['AMT_ANNUITY', 'AMT_CREDIT_MAX_OVERDUE'], inplace=True)
    df_test.drop(columns=['AMT_ANNUITY', 'AMT_CREDIT_MAX_OVERDUE'], inplace=True)
    imputer = SimpleImputer(strategy='mean')
    col_to_impute = ['AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM']
    df_train[col_to_impute] = imputer.fit_transform(df_train[col_to_impute])
    df_test[col_to_impute] = imputer.transform(df_test[col_to_impute])

    return df_train, df_test

### 2.Feature Engineering

In [128]:
def create_bureau_balance(df):

    #label encode STATUS
    dict_for_status = { 'C': 12, 'X': 11, '0': 10, '1': 5, '2': 4, '3': 3, '4': 2, '5': 1}
    df['STATUS'] = df['STATUS'].map(dict_for_status)

    #converting months to positive
    #weighing the status with the months_balance
    df['MONTHS_BALANCE'] = np.abs(df['MONTHS_BALANCE'])
    df['WEIGHTED_STATUS'] = df.STATUS / (df.MONTHS_BALANCE + 1)

    #sorting the bureau_balance in ascending order of month and by the bureau SK_ID
    df = df.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=[0, 0])
    #we will do exponential weighted average on the encoded status
    #this is because if a person had a bad status 2 years ago, it should be given less weightage today
    # we keep the latent variable alpha = 0.8 
    #doing this for both weighted status and the status itself
    df['EXP_WEIGHTED_STATUS'] = df.groupby('SK_ID_BUREAU')['WEIGHTED_STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())
    df['EXP_ENCODED_STATUS'] = df.groupby('SK_ID_BUREAU')['STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())    

    df['MONTHS_BALANCE'] = df['MONTHS_BALANCE'] // 12

    #defining our aggregations
    aggregations_basic = {
        'MONTHS_BALANCE' : ['mean','max'],
        'STATUS' : ['mean','max','first'],
        'WEIGHTED_STATUS' : ['mean','sum','first'],
        'EXP_ENCODED_STATUS' : ['last'],
        'EXP_WEIGHTED_STATUS' : ['last']}

    #we will be finding aggregates for each year too
    aggregations_for_year = {
        'STATUS' : ['mean','max','last','first'],
        'WEIGHTED_STATUS' : ['mean','max', 'first','last'],
        'EXP_WEIGHTED_STATUS' : ['last'],
        'EXP_ENCODED_STATUS' : ['last'] }

    #aggregating over whole dataset first
    aggregated_df = df.groupby(['SK_ID_BUREAU']).agg(aggregations_basic)
    aggregated_df.columns = ['_'.join(ele).upper() for ele in aggregated_df.columns]

    #aggregating some of the features separately for latest 2 years
    aggregated_df_years = pd.DataFrame()
    for year in range(2):
        year_group = df[df['MONTHS_BALANCE'] == year].groupby('SK_ID_BUREAU').agg(aggregations_for_year)
        year_group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in year_group.columns]

        if year == 0:
            aggregated_df_years = year_group
        else:
            aggregated_df_years = aggregated_df_years.merge(year_group, on = 'SK_ID_BUREAU', how = 'outer')

    #aggregating for rest of the years
    aggregated_df_rest_years = df[df.MONTHS_BALANCE > year].groupby(['SK_ID_BUREAU']).agg(aggregations_for_year)
    aggregated_df_rest_years.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in aggregated_df_rest_years.columns]

    #merging with rest of the years
    aggregated_df_years = aggregated_df_years.merge(aggregated_df_rest_years, on = 'SK_ID_BUREAU', how = 'outer')
    aggregated_df = aggregated_df.merge(aggregated_df_years, on = 'SK_ID_BUREAU', how = 'inner')

    #filling the missing values obtained after aggregations with 0
    aggregated_df.fillna(0, inplace = True)

    return aggregated_df

def create_bureau(df):
    #create features based on domain
    df['CREDIT_DURATION'] = np.abs(df['DAYS_CREDIT'] - df['DAYS_CREDIT_ENDDATE'])
    df['FLAG_OVERDUE_RECENT'] = [0 if ele == 0 else 1 for ele in df['CREDIT_DAY_OVERDUE']]
    # df['MAX_AMT_OVERDUE_DURATION_RATIO'] = df['AMT_CREDIT_MAX_OVERDUE'] / (df['CREDIT_DURATION'] + 0.00001)
    df['CURRENT_AMT_OVERDUE_DURATION_RATIO'] = df['AMT_CREDIT_SUM_OVERDUE'] / (df['CREDIT_DURATION'] + 0.00001)
    df['AMT_OVERDUE_DURATION_LEFT_RATIO'] = df['AMT_CREDIT_SUM_OVERDUE'] / (df['DAYS_CREDIT_ENDDATE'] + 0.00001)
    # df['CNT_PROLONGED_MAX_OVERDUE_MUL'] = df['CNT_CREDIT_PROLONG'] * df['AMT_CREDIT_MAX_OVERDUE']
    df['CNT_PROLONGED_DURATION_RATIO'] = df['CNT_CREDIT_PROLONG'] / (df['CREDIT_DURATION'] + 0.00001)
    df['CURRENT_DEBT_TO_CREDIT_RATIO'] = df['AMT_CREDIT_SUM_DEBT'] / (df['AMT_CREDIT_SUM'] + 0.00001)
    df['CURRENT_CREDIT_DEBT_DIFF'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_DEBT']
    # df['AMT_ANNUITY_CREDIT_RATIO'] = df['AMT_ANNUITY'] / (df['AMT_CREDIT_SUM'] + 0.00001)
    df['CREDIT_ENDDATE_UPDATE_DIFF'] = np.abs(df['DAYS_CREDIT_UPDATE'] - df['DAYS_CREDIT_ENDDATE'])

    #now we will be aggregating the bureau_merged df with respect to 'SK_ID_CURR' so as to merge it with application_train later    
    #firstly we will aggregate the columns based on the category of CREDIT_ACTIVE
    aggregations_CREDIT_ACTIVE = {
                    'DAYS_CREDIT' : ['mean','min','max','last'],
                    'CREDIT_DAY_OVERDUE' : ['mean','max'],
                    'DAYS_CREDIT_ENDDATE' : ['mean','max'],
                    'DAYS_ENDDATE_FACT' : ['mean','min'],
                    # 'AMT_CREDIT_MAX_OVERDUE': ['max','sum'],
                    'CNT_CREDIT_PROLONG': ['max','sum'],
                    'AMT_CREDIT_SUM' : ['sum','max'],
                    'AMT_CREDIT_SUM_DEBT': ['sum'],
                    'AMT_CREDIT_SUM_LIMIT': ['max','sum'],
                    'AMT_CREDIT_SUM_OVERDUE': ['max','sum'],
                    'DAYS_CREDIT_UPDATE' : ['mean','min'],
                    # 'AMT_ANNUITY' : ['mean','sum','max'],
                    'CREDIT_DURATION' : ['max','mean'],
                    'FLAG_OVERDUE_RECENT': ['sum'],
                    # 'MAX_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                    'CURRENT_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                    'AMT_OVERDUE_DURATION_LEFT_RATIO' : ['max', 'mean'],
                    # 'CNT_PROLONGED_MAX_OVERDUE_MUL' : ['mean','max'],
                    'CNT_PROLONGED_DURATION_RATIO' : ['mean', 'max'],
                    'CURRENT_DEBT_TO_CREDIT_RATIO' : ['mean', 'min'],
                    'CURRENT_CREDIT_DEBT_DIFF' : ['mean','min'],
                    # 'AMT_ANNUITY_CREDIT_RATIO' : ['mean','max','min'],
                    'CREDIT_ENDDATE_UPDATE_DIFF' : ['max','min'],
                    'STATUS_MEAN' : ['mean', 'max'],
                    'WEIGHTED_STATUS_MEAN' : ['mean', 'max']
                        }

    #we saw from EDA that the two most common type of CREDIT ACTIVE were 'Closed' and 'Active'.
    #So we will aggregate them two separately and the remaining categories separately.
    categories_to_aggregate_on = ['Closed','Active']
    df_aggregated_credit = pd.DataFrame()
    for i, status in enumerate(categories_to_aggregate_on):
        group = df[df['CREDIT_ACTIVE'] == status].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
        group.columns = ['_'.join(ele).upper() + '_CREDITACTIVE_' + status.upper() for ele in group.columns]

        if i==0:
            df_aggregated_credit = group
        else:
            df_aggregated_credit = df_aggregated_credit.merge(group, on = 'SK_ID_CURR', how = 'outer')
    
    #aggregating for remaining categories
    df_aggregated_credit_rest = df[(df['CREDIT_ACTIVE'] != 'Active') & 
                                                            (df['CREDIT_ACTIVE'] != 'Closed')].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
    df_aggregated_credit_rest.columns = ['_'.join(ele).upper() + 'CREDIT_ACTIVE_REST' for ele in df_aggregated_credit_rest.columns]

    #merging with other categories
    df_aggregated_credit = df_aggregated_credit.merge(df_aggregated_credit_rest, on = 'SK_ID_CURR', how = 'outer')

    #Encoding the categorical columns in one-hot form
    currency_ohe = pd.get_dummies(df['CREDIT_CURRENCY'], prefix = 'CURRENCY')
    credit_active_ohe = pd.get_dummies(df['CREDIT_ACTIVE'], prefix = 'CREDIT_ACTIVE')
    credit_type_ohe = pd.get_dummies(df['CREDIT_TYPE'], prefix = 'CREDIT_TYPE')

    #merging the one-hot encoded columns
    df = pd.concat([df.drop(['CREDIT_CURRENCY','CREDIT_ACTIVE','CREDIT_TYPE'], axis = 1), 
                                currency_ohe, credit_active_ohe, credit_type_ohe], axis = 1)

    #aggregating the bureau_merged over all the columns
    df_aggregated = df.drop('SK_ID_BUREAU', axis = 1).groupby('SK_ID_CURR').agg('mean')
    df_aggregated.columns = [ele if 'TARGET' in ele else ele + '_MEAN_OVERALL' for ele in df_aggregated.columns]
    #merging it with aggregates over categories
    df_aggregated = df_aggregated.merge(df_aggregated_credit, on = 'SK_ID_CURR', how = 'outer')

    return df_aggregated



### 3.Main Function


In [129]:
def bureau_and_balance():
    bureau_balance = pd.read_csv('data/bureau_balance.csv')
    bureau = pd.read_csv('data/bureau.csv')
    target = pd.read_csv('data/target.csv')
    
    #merge bureau_balance with bureau to get SK_ID_CURR
    bb_target = bureau_balance.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on='SK_ID_BUREAU', how='left')
    #merge bureau_balance with target
    bb_target = bb_target.merge(target, on='SK_ID_CURR', how='left')
    bb_target = bb_target.drop('SK_ID_CURR', axis=1)

    #split bb into train and test
    bb_train = bb_target[bb_target['TARGET'].notnull()]
    bb_test = bb_target[bb_target['TARGET'].isnull()]
    bb_train = bb_train.drop('TARGET', axis=1)
    bb_test = bb_test.drop('TARGET', axis=1)

    #create fetures for bb
    bb_train = create_bureau_balance(bb_train)
    bb_test = create_bureau_balance(bb_test)

    #merge bureau with target
    bureau_target = bureau.merge(target, on = 'SK_ID_CURR', how = 'left')
    bureau_train = bureau_target[bureau_target['TARGET'].notnull()]
    bureau_test = bureau_target[bureau_target['TARGET'].isnull()]
    
    
    #merge train, test with bureau
    train = bureau_train.merge(bb_train, on = 'SK_ID_BUREAU', how = 'left')
    test = bureau_test.merge(bb_test, on = 'SK_ID_BUREAU', how = 'left')

    # cleaning
    # train, test = fill_missing_bureau(train, test, bb_train)
    train, test = lazy_fill_missing_bureau(train, test)
    train, test = clipping_outliers_app(train, test)
    # FE 
    train = create_bureau(train)
    test = create_bureau(test)
    # return train, test
    #merge train and test
    df = pd.concat([train, test], axis = 0)

    # # clean some invalid data after FE
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.reset_index(inplace=True)
    train = df[df['TARGET'].notna()]
    test = df[df['TARGET'].isna()]
    train = select_feature(train)
    df = pd.concat([train, test[train.columns]], axis=0)
    if 'TARGET' in df.columns: df.drop(columns='TARGET', inplace=True)
    # sr_na = df.isna().sum()
    # sr_na = sr_na / len(df) * 100
    # col_to_drop = ['TARGET'] + sr_na[sr_na > 50].index.to_list()
    # df.drop(columns=col_to_drop, inplace=True)

    # train = df[df['TARGET'].notna()]
    # test = df[df['TARGET'].isna()]

    
    return df

### TEMPORARY

In [130]:
def bureau_and_balance_temp():
    bureau = pd.read_csv(r'data/bureau.csv')
    bb = pd.read_csv(r'data/bureau_balance.csv')

    # Credit duration and credit/account end date difference
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    
    # Credit to debt ratio and difference
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']
    bureau['BUREAU_CREDIT_FACT_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_ENDDATE_FACT']
    bureau['BUREAU_CREDIT_ENDDATE_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
    bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']

    # CREDIT_DAY_OVERDUE :
    bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
    bureau['BUREAU_IS_DPD_OVER120'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 120 else 0)

    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size', 'mean']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']

    #Status of Credit Bureau loan during the month
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean', 'min'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean', 'sum'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'SK_ID_BUREAU': ['count'],
        'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
        'ENDDATE_DIF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_FACT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_ENDDATE_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_DEBT_RATIO': ['min', 'max', 'mean'],
        'DEBT_CREDIT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_IS_DPD': ['mean', 'sum'],
        'BUREAU_IS_DPD_OVER120': ['mean', 'sum']
        }

    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    bureau_agg.reset_index(inplace=True)
    print('"Bureau/Bureau Balance" final shape:', bureau_agg.shape)
    
    return bureau_agg

In [131]:
# df = bureau_bb()
# df

## III. Credit Card Balance

### 1.Clean Data

In [132]:
# #MinhHuong - fixed, add ewm
# def lazy_fill_missing_cc(dataframe_train, dataframe_test):
#     col_num = dataframe_train.select_dtypes('number').columns.drop('TARGET')
#     for col in col_num:
#         median_value = dataframe_train[col].median()
#         dataframe_train[col] = dataframe_train[col].fillna(median_value)
#         dataframe_test[col] = dataframe_test[col].fillna(median_value)

#     col_object = dataframe_train.select_dtypes('object').columns
#     for col in col_object:
#         mode_value = dataframe_train[col].mode()[0]
#         dataframe_train[col] = dataframe_train[col].fillna(mode_value)
#         dataframe_test[col] = dataframe_test[col].fillna(mode_value)
#     return dataframe_train, dataframe_test

def missing_values(df_train, df_test):
    drawings_cols = ['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT',
                'AMT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT',
                'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT']

    # fill 0 for AMT_BALANCE = 0 -> no transactions are made
    df_train.loc[df_train['AMT_BALANCE'] == 0, drawings_cols].fillna(0)
    df_test.loc[df_test['AMT_BALANCE'] == 0, drawings_cols].fillna(0)

    # Fill remaining missing values in drawings_cols with 0 if both AMT_DRAWINGS_CURRENT and CNT_DRAWINGS_CURRENT are 0
    df_train.loc[(df_train['AMT_DRAWINGS_CURRENT'] == 0) & (df_train['CNT_DRAWINGS_CURRENT'] == 0), drawings_cols].fillna(0)
    df_test.loc[(df_test['AMT_DRAWINGS_CURRENT'] == 0) & (df_test['CNT_DRAWINGS_CURRENT'] == 0), drawings_cols].fillna(0)

    # Fill AMT_PAYMENT_CURRENT
    mean_payment = df_train['AMT_PAYMENT_CURRENT'].mean()
    df_train['AMT_PAYMENT_CURRENT'].fillna(mean_payment, inplace=True)
    df_test['AMT_PAYMENT_CURRENT'].fillna(mean_payment, inplace=True)

    #fill AMT_INST_MIN_REGULARITY
    payment_ratio = df_train['AMT_INST_MIN_REGULARITY'].dropna() / df_train['AMT_CREDIT_LIMIT_ACTUAL']
    median_ratio = payment_ratio.median()

    missing_mask_train = df_train['AMT_INST_MIN_REGULARITY'].isna()
    missing_mask_test = df_test['AMT_INST_MIN_REGULARITY'].isna()
    df_train.loc[missing_mask_train, 'AMT_INST_MIN_REGULARITY'] = df_train.loc[missing_mask_train, 'AMT_CREDIT_LIMIT_ACTUAL'] * median_ratio
    df_test.loc[missing_mask_test, 'AMT_INST_MIN_REGULARITY'] = df_test.loc[missing_mask_test, 'AMT_CREDIT_LIMIT_ACTUAL'] * median_ratio

    #fill CNT_INSTALMENT_MATURE_CUM
    mean_cnt = df_train['CNT_INSTALMENT_MATURE_CUM'].mean().round()
    df_train['CNT_INSTALMENT_MATURE_CUM'].fillna(mean_cnt, inplace=True)
    df_test['CNT_INSTALMENT_MATURE_CUM'].fillna(mean_cnt, inplace=True)

    return df_train, df_test

### 2.Feature Engineering


In [133]:
def preprocessing_and_feature_engineering(df):
    #drop one abruptly large value in AMT_PAYMENT_CURRENT
    df.drop(df[df['AMT_PAYMENT_CURRENT'] > 4000000].index, inplace=True)

    # feature engineering
    # making the MONTHS_BALANCE Positive
    df['MONTHS_BALANCE'] = np.abs(df['MONTHS_BALANCE'])

    #sorting according to the month of status from oldest to latest, for rolling computations
    df = df.sort_values(by = ['SK_ID_PREV','MONTHS_BALANCE'], ascending = [1,0])

    df['AMT_DRAWINGS_SUM'] = df[['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT']].sum(axis=1)
    df['CNT_DRAWINGS_SUM'] = df[['CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT']].sum(axis=1)

    df['BALANCE_LIMIT_RATIO'] = df['AMT_BALANCE'] / (df['AMT_CREDIT_LIMIT_ACTUAL'] + 0.00001)

    df['MIN_PAYMENT_RATIO'] = df['AMT_PAYMENT_CURRENT'] / (df['AMT_INST_MIN_REGULARITY'] + 0.0001)
    df['PAYMENT_MIN_DIFF'] = df['AMT_PAYMENT_CURRENT'] - df['AMT_INST_MIN_REGULARITY']
    df['PAYMENT_MIN_DIFF'] = df['AMT_PAYMENT_TOTAL_CURRENT'] - df['AMT_INST_MIN_REGULARITY']
    df['MIN_PAYMENT_TOTAL_RATIO'] = df['AMT_PAYMENT_TOTAL_CURRENT'] / (df['AMT_INST_MIN_REGULARITY'] +0.00001)

    df['AMT_INTEREST_RECEIVABLE'] = df['AMT_TOTAL_RECEIVABLE'] - df['AMT_RECEIVABLE_PRINCIPAL']

    df['SK_DPD_RATIO'] = df['SK_DPD'] / (df['SK_DPD_DEF'] + 0.00001)

    # calculating the rolling Exponential Weighted Moving Average over months for certain features
    rolling_col = [
        'AMT_BALANCE',
        'AMT_CREDIT_LIMIT_ACTUAL',
        'AMT_RECEIVABLE_PRINCIPAL',
        'AMT_RECIVABLE',
        'AMT_TOTAL_RECEIVABLE',
        'AMT_DRAWINGS_SUM',
        'BALANCE_LIMIT_RATIO',
        'CNT_DRAWINGS_SUM',
        'MIN_PAYMENT_RATIO',
        'PAYMENT_MIN_DIFF',
        'MIN_PAYMENT_TOTAL_RATIO',
        'AMT_INTEREST_RECEIVABLE',
        'SK_DPD_RATIO']

    exp_weighted_col = ['EXP_' + ele for ele in rolling_col]
    df[exp_weighted_col] = df.groupby(['SK_ID_CURR','SK_ID_PREV'])[rolling_col].transform(lambda x: x.ewm(alpha = 0.7).mean())

    #performing overall aggregations over SK_ID_PREV
    overall_aggregations = {
    'SK_ID_CURR' : ['first'],
    'MONTHS_BALANCE': ['max'],
    'AMT_BALANCE' : ['sum','mean','max'],
    'AMT_CREDIT_LIMIT_ACTUAL' : ['sum','mean','max'],
    'AMT_DRAWINGS_ATM_CURRENT' : ['sum','max'],
    'AMT_DRAWINGS_CURRENT' : ['sum','max'],
    'AMT_DRAWINGS_OTHER_CURRENT' : ['sum','max'],
    'AMT_DRAWINGS_POS_CURRENT' : ['sum','max'],
    'AMT_INST_MIN_REGULARITY' : ['mean','min','max'],
    'AMT_PAYMENT_CURRENT' : ['mean','min','max'],
    'AMT_PAYMENT_TOTAL_CURRENT' : ['mean','min','max'],
    'AMT_RECEIVABLE_PRINCIPAL' : ['sum','mean','max'],
    'AMT_RECIVABLE' : ['sum','mean','max'],
    'AMT_TOTAL_RECEIVABLE' : ['sum','mean','max'],
    'CNT_DRAWINGS_ATM_CURRENT' : ['sum','max'],
    'CNT_DRAWINGS_CURRENT' : ['sum','max'],
    'CNT_DRAWINGS_OTHER_CURRENT' : ['sum','max'],
    'CNT_DRAWINGS_POS_CURRENT' : ['sum','max'],
    'CNT_INSTALMENT_MATURE_CUM' : ['sum','max','min'],
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],

    'AMT_DRAWINGS_SUM' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'CNT_DRAWINGS_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'],
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],

    'EXP_AMT_BALANCE' : ['last'],
    'EXP_AMT_CREDIT_LIMIT_ACTUAL' : ['last'],
    'EXP_AMT_RECEIVABLE_PRINCIPAL' : ['last'],
    'EXP_AMT_RECIVABLE' : ['last'],
    'EXP_AMT_TOTAL_RECEIVABLE' : ['last'],
    'EXP_AMT_DRAWINGS_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWINGS_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last'],
    }

    cc_balance_aggregated_overall = df.groupby('SK_ID_PREV').agg(overall_aggregations)
    cc_balance_aggregated_overall.columns = ['_'.join(ele).upper() for ele in cc_balance_aggregated_overall.columns]
    cc_balance_aggregated_overall.rename(columns = {'SK_ID_CURR_FIRST' : 'SK_ID_CURR'}, inplace = True)

    # aggregating for different categories
    aggregations_for_categories = {
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'AMT_DRAWINGS_SUM' : ['sum','max'],
    'CNT_DRAWINGS_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'],
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],
    'EXP_AMT_DRAWINGS_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWINGS_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last']
}

    contract_status_categories = ['Active','Completed']
    cc_balance_aggregated_categories = pd.DataFrame()
    for i, contract_type in enumerate(contract_status_categories):
        group = df[df['NAME_CONTRACT_STATUS'] == contract_type].groupby('SK_ID_PREV').agg(aggregations_for_categories)
        group.columns = ['_'.join(ele).upper() + '_' + contract_type.upper() for ele in group.columns]
        if i == 0:
            cc_balance_aggregated_categories = group
        else:
            cc_balance_aggregated_categories = cc_balance_aggregated_categories.merge(group, on = 'SK_ID_PREV', how = 'outer')

    # aggregating over SK_ID_PREV for rest of the categories
    cc_balance_aggregated_categories_rest = df[(df['NAME_CONTRACT_STATUS'] != 'Active') &
                                    (df.NAME_CONTRACT_STATUS != 'Completed')].groupby('SK_ID_PREV').agg(aggregations_for_categories)
    cc_balance_aggregated_categories_rest.columns = ['_'.join(ele).upper() + '_REST' for ele in cc_balance_aggregated_categories_rest.columns]

    #merging all the categorical aggregations
    cc_balance_aggregated_categories = cc_balance_aggregated_categories.merge(cc_balance_aggregated_categories_rest, on = 'SK_ID_PREV', how = 'outer')

    #merging all the aggregations
    cc_aggregated = cc_balance_aggregated_overall.merge(cc_balance_aggregated_categories, on = 'SK_ID_PREV', how = 'outer')

    #one-hot encoding the categorical column NAME_CONTRACT_STATUS
    name_contract_dummies = pd.get_dummies(df.NAME_CONTRACT_STATUS, prefix='CONTRACT')
    contract_names = name_contract_dummies.columns.tolist()

    #merging the one-hot encoded feature with original table
    df = pd.concat([df, name_contract_dummies], axis=1)
    #aggregating over SK_ID_PREV the one-hot encoded columns
    aggregated_cc_contract = df[['SK_ID_PREV'] + contract_names].groupby('SK_ID_PREV').mean()

    #merging with the aggregated table
    cc_aggregated = cc_aggregated.merge(aggregated_cc_contract, on = 'SK_ID_PREV', how = 'outer')

    #aggregate on SK_ID_CURR
    #from EDA, since most of the SK_ID_CURR had only 1 credit card, so for aggregations, we will take the means
    cc_aggregated = cc_aggregated.groupby('SK_ID_CURR', as_index = False).mean()

    return cc_aggregated

### 3.Main Function

In [151]:


def credit_card_balance():
    # Load data
    cc = pd.read_csv('data/credit_card_balance.csv')
    target = pd.read_csv('data/target.csv')

    # Set target index
    target.set_index('SK_ID_CURR', inplace=True)

    # Merge with target and split train/test
    cc_agg = cc.merge(target, on='SK_ID_CURR', how='left')
    cc_train = cc_agg[~cc_agg['TARGET'].isnull()]
    cc_test = cc_agg[cc_agg['TARGET'].isnull()]

    # clean
    # cc_train, cc_test = missing_values(cc_train, cc_test)
    cc_train, cc_test = missing_values(cc_train, cc_test)
    cc_train, cc_test = clipping_outliers_app(cc_train, cc_test)
    # Process
    cc_train_processed = preprocessing_and_feature_engineering(cc_train)
    cc_train_processed = cc_train_processed.replace([np.inf, -np.inf], np.nan)

    cc_test_processed = preprocessing_and_feature_engineering(cc_test)
    cc_test_processed = cc_test_processed.replace([np.inf, -np.inf], np.nan)
    # Combine train and test sets
    processed_credit_card_balance = pd.concat([cc_train_processed, cc_test_processed], axis=0)
    # print(11)
    # train = processed_credit_card_balance[processed_credit_card_balance['TARGET'].notna()]
    # test = processed_credit_card_balance[processed_credit_card_balance['TARGET'].isna()]
    # train = select_feature(train)
    # processed_credit_card_balance = pd.concat([train, test[train.columns]], axis=0)
    print('"Credit Card Balance" final shape:', processed_credit_card_balance.shape)

    return processed_credit_card_balance


### TEMPORARY

In [135]:
def credit_card_balance_temp():    
    cc = pd.read_csv(r'data/credit_card_balance.csv')

    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)

    # Amount used from limit
    cc['LIMIT_USE'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
    # Current payment / Min payment
    cc['PAYMENT_DIV_MIN'] = cc['AMT_PAYMENT_CURRENT'] / cc['AMT_INST_MIN_REGULARITY']
    # Late payment <-- 'CARD_IS_DPD'
    cc['LATE_PAYMENT'] = cc['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    # How much drawing of limit
    cc['DRAWING_LIMIT_RATIO'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / cc['AMT_CREDIT_LIMIT_ACTUAL']

    cc['CARD_IS_DPD_UNDER_120'] = cc['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    cc['CARD_IS_DPD_OVER_120'] = cc['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # General aggregations
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

    # Last month balance of each credit card application
    last_ids = cc.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    last_months_df = cc[cc.index.isin(last_ids)]
    cc_agg = group_and_merge(last_months_df,cc_agg,'CC_LAST_', {'AMT_BALANCE': ['mean', 'max']})

    CREDIT_CARD_TIME_AGG = {
        'AMT_BALANCE': ['mean', 'max'],
        'LIMIT_USE': ['max', 'mean'],
        'AMT_CREDIT_LIMIT_ACTUAL':['max'],
        'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],
        'AMT_DRAWINGS_CURRENT': ['max', 'sum'],
        'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],
        'AMT_INST_MIN_REGULARITY': ['max', 'mean'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['max','sum'],
        'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],
        'CNT_DRAWINGS_ATM_CURRENT': ['max','sum', 'mean'],
        'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
        'CNT_DRAWINGS_POS_CURRENT': ['mean'],
        'SK_DPD': ['mean', 'max', 'sum'],
        'LIMIT_USE': ['min', 'max'],
        'DRAWING_LIMIT_RATIO': ['min', 'max'],
        'LATE_PAYMENT': ['mean', 'sum'],
        'CARD_IS_DPD_UNDER_120': ['mean', 'sum'],
        'CARD_IS_DPD_OVER_120': ['mean', 'sum']
    }

    for months in [12, 24, 48]:
        cc_prev_id = cc[cc['MONTHS_BALANCE'] >= -months]['SK_ID_PREV'].unique()
        cc_recent = cc[cc['SK_ID_PREV'].isin(cc_prev_id)]
        prefix = 'INS_{}M_'.format(months)
        cc_agg = group_and_merge(cc_recent, cc_agg, prefix, CREDIT_CARD_TIME_AGG)


    print('"Credit Card Balance" final shape:', cc_agg.shape)
    return cc_agg

## IV. Installments Payments

### 1.Clean Data

In [ ]:
# MinhHuong - fix feature engineering (add ewm), bổ sung aggregation

def missing_values_ins(df_train, df_test):
    # Remove null from AMT_INSTALMENT and SK_ID_CURR
    df_train.dropna(subset=['SK_ID_CURR'], inplace = True)
    df_test.dropna(subset=['SK_ID_CURR'], inplace = True)
    df_train.dropna(subset=['AMT_INSTALMENT'], inplace = True)
    df_test.dropna(subset=['AMT_INSTALMENT'], inplace = True)

    # null in DAY_ENTRY_PAYMENT -> null in AMT_PAYMENT null: unpaid loan
    max_days = df_train['DAYS_ENTRY_PAYMENT'].max() + 1
    df_train['DAYS_ENTRY_PAYMENT'].fillna(max_days, inplace = True) # fill max value -> latest payment
    df_test['DAYS_ENTRY_PAYMENT'].fillna(max_days, inplace = True)

    df_train['AMT_PAYMENT'].fillna(0, inplace=True) # fill 0 -> unpaid
    df_test['AMT_PAYMENT'].fillna(0, inplace=True)

    return df_train, df_test

### 2.Feature Engineering

In [ ]:
def feature_engineering_ins(df):

    # new features
    df['AMT_PAYMENT_RATIO'] = df['AMT_PAYMENT'] / df['AMT_INSTALMENT']
    df['FLAG_LOW_PAYMENT_RATIO'] = (df['AMT_PAYMENT_RATIO'] < 0.5).astype(int)

    df['AMT_PAYMENT_DIFF'] = df['AMT_INSTALMENT'] - df['AMT_PAYMENT']
    df['LESS_PAYMENT_FLAG'] = (df['AMT_PAYMENT'] < df['AMT_INSTALMENT']).astype(int)

    df['DUE_FLAG'] = (df['DAYS_ENTRY_PAYMENT'] > df['DAYS_INSTALMENT']).astype(int)
    df['DAYS_PAYMENT_RATIO'] = df['DAYS_INSTALMENT'] / df['DAYS_ENTRY_PAYMENT']
    df['DAYS_PAYMENT_DIFF'] = df['DAYS_INSTALMENT'] - df['DAYS_ENTRY_PAYMENT']

    # add ewm
    df['EXP_DAYS_PAYMENT_RATIO'] = df['DAYS_PAYMENT_RATIO'].transform(lambda x: x.ewm(alpha = 0.5).mean())
    df['EXP_DAYS_PAYMENT_DIFF'] = df['DAYS_PAYMENT_DIFF'].transform(lambda x: x.ewm(alpha = 0.5).mean())
    df['EXP_AMT_PAYMENT_RATIO'] = df['AMT_PAYMENT_RATIO'].transform(lambda x: x.ewm(alpha = 0.5).mean())
    df['EXP_AMT_PAYMENT_DIFF'] = df['AMT_PAYMENT_DIFF'].transform(lambda x: x.ewm(alpha = 0.5).mean())

    # Aggregate features per SK_ID_PREV
    aggregate = {
    'NUM_INSTALMENT_VERSION' : ['mean','sum'],
    'NUM_INSTALMENT_NUMBER' : ['max'],
    'DAYS_INSTALMENT' : ['max','min'],
    'DAYS_ENTRY_PAYMENT' : ['max','min'],
    'AMT_INSTALMENT' : ['mean', 'sum', 'max'],
    'AMT_PAYMENT' : ['mean', 'sum', 'max'],
    'DAYS_PAYMENT_RATIO' : ['mean', 'min','max'],
    'DAYS_PAYMENT_DIFF' : ['mean','min','max'],
    'AMT_PAYMENT_RATIO' : ['mean','min','max'],
    'AMT_PAYMENT_DIFF' : ['mean','min','max'],
    'FLAG_LOW_PAYMENT_RATIO': ['mean', 'sum', 'last'],
    'LESS_PAYMENT_FLAG': ['mean', 'sum'],
    'DUE_FLAG': ['mean', 'sum'],
    'EXP_DAYS_PAYMENT_RATIO' : ['last'],
    'EXP_DAYS_PAYMENT_DIFF' : ['last'],
    'EXP_AMT_PAYMENT_RATIO' : ['last'],
    'EXP_AMT_PAYMENT_DIFF' : ['last']
    }

    limited_period_aggregations = {
    'NUM_INSTALMENT_VERSION' : ['mean','sum'],
    'AMT_PAYMENT_RATIO': ['mean', 'min'],
    'AMT_PAYMENT_DIFF': ['mean', 'sum'],
    'LESS_PAYMENT_FLAG': ['mean', 'sum'],
    'DUE_FLAG': ['mean', 'sum'],
    'DAYS_PAYMENT_RATIO': ['mean', 'max'],
    'DAYS_PAYMENT_DIFF': ['mean', 'sum'],
    'EXP_DAYS_PAYMENT_RATIO' : ['last'],
    'EXP_DAYS_PAYMENT_DIFF' : ['last'],
    'EXP_AMT_PAYMENT_RATIO' : ['last'],
    'EXP_AMT_PAYMENT_DIFF' : ['last']
    }A

    special_aggregations = {
    'AMT_PAYMENT_RATIO': ['mean', 'min'],
    'AMT_PAYMENT_DIFF': ['mean', 'sum'],
    'LESS_PAYMENT_FLAG': ['mean', 'sum'],
    'DUE_FLAG': ['mean', 'sum'],
    'DAYS_PAYMENT_RATIO': ['mean', 'max'],
    'DAYS_PAYMENT_DIFF': ['mean', 'sum'],
    'EXP_DAYS_PAYMENT_RATIO' : ['last'],
    'EXP_DAYS_PAYMENT_DIFF' : ['last'],
    'EXP_AMT_PAYMENT_RATIO' : ['last'],
    'EXP_AMT_PAYMENT_DIFF' : ['last']}

    # Aggregates over last 60, 90, 180 & 365 days
    time = {'60': -60, '90': -90, '180': -180, '365': -365}
    aggregations_time = []

    for key, value in time.items():
        target = df[df['DAYS_INSTALMENT'] > value]
        time_agg = target.groupby('SK_ID_PREV').agg(limited_period_aggregations)
        time_agg.columns = ['_'.join(ele).upper() + f'_LAST_{key}_DAYS' for ele in time_agg.columns]
        aggregations_time.append(time_agg)

    aggregations_time = pd.concat(aggregations_time, axis=1)

    # First 5 installments aggregation
    group_first_5_instalments = df.groupby('SK_ID_PREV', as_index = False).head(5).groupby('SK_ID_PREV').agg(limited_period_aggregations)
    group_first_5_instalments.columns = ['_'.join(ele).upper() + '_FIRST_5_INSTALLMENTS' for ele in group_first_5_instalments.columns]

    #Aggregates over NUM_INSTALMENT_NUMBER at 4 installments
    instalment_4 = df[df['NUM_INSTALMENT_NUMBER'] == 4]
    aggregations_instalment_4 = instalment_4.groupby('SK_ID_PREV').agg(special_aggregations)
    aggregations_instalment_4.columns = ['_'.join(col).upper() + '_INSTAL_4' for col in aggregations_instalment_4.columns]

    # aggregates over all installments that were past due
    past_due = df[df['DUE_FLAG'] == 1]
    aggregations_past_due = past_due.groupby('SK_ID_PREV').agg(special_aggregations)
    aggregations_past_due.columns = ['_'.join(col).upper() + '_PAST_DUE' for col in aggregations_past_due.columns]

    #overall aggregation of installments_payments over SK_ID_PREV
    group_overall = df.groupby(['SK_ID_PREV','SK_ID_CURR'], as_index = False).agg(aggregate)
    group_overall.columns = ['_'.join(ele).upper() for ele in group_overall.columns]
    group_overall.rename(columns = {'SK_ID_PREV_': 'SK_ID_PREV','SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)

    # merge
    installments_payments_agg_prev = group_overall.merge(aggregations_time, on = 'SK_ID_PREV', how = 'outer')
    installments_payments_agg_prev = installments_payments_agg_prev.merge(group_first_5_instalments, on = 'SK_ID_PREV', how = 'outer')
    installments_payments_agg_prev = installments_payments_agg_prev.merge(aggregations_instalment_4, on = 'SK_ID_PREV', how = 'outer')
    installments_payments_agg_prev = installments_payments_agg_prev.merge(aggregations_past_due, on = 'SK_ID_PREV', how = 'outer')

    #aggregating over SK_ID_CURR
    main_features_aggregations = {
    'NUM_INSTALMENT_VERSION_MEAN' : ['mean'],
    'NUM_INSTALMENT_VERSION_SUM' : ['mean'],
    'NUM_INSTALMENT_NUMBER_MAX' : ['mean','sum','max'],
    'AMT_INSTALMENT_MEAN' : ['mean','sum','max'],
    'AMT_INSTALMENT_SUM' : ['mean','sum','max'],
    'AMT_INSTALMENT_MAX' : ['mean'],
    'AMT_PAYMENT_MEAN' : ['mean','sum','max'],
    'AMT_PAYMENT_SUM' : ['mean','sum','max'],
    'AMT_PAYMENT_MAX' : ['mean'],
    'DAYS_PAYMENT_RATIO_MEAN' : ['mean','min','max'],
    'DAYS_PAYMENT_RATIO_MIN' : ['mean','min'],
    'DAYS_PAYMENT_RATIO_MAX' : ['mean','max'],
    'DAYS_PAYMENT_DIFF_MEAN' : ['mean','min','max'],
    'DAYS_PAYMENT_DIFF_MIN' : ['mean','min'],
    'DAYS_PAYMENT_DIFF_MAX' : ['mean','max'],
    'AMT_PAYMENT_RATIO_MEAN' : ['mean', 'min','max'],
    'AMT_PAYMENT_RATIO_MIN' : ['mean','min'],
    'AMT_PAYMENT_RATIO_MAX' : ['mean','max'],
    'AMT_PAYMENT_DIFF_MEAN' : ['mean','min','max'],
    'AMT_PAYMENT_DIFF_MIN' : ['mean','min'],
    'AMT_PAYMENT_DIFF_MAX' : ['mean','max'],
    'EXP_DAYS_PAYMENT_RATIO_LAST' : ['mean'],
    'EXP_DAYS_PAYMENT_DIFF_LAST' : ['mean'],
    'EXP_AMT_PAYMENT_RATIO_LAST' : ['mean'],
    'EXP_AMT_PAYMENT_DIFF_LAST' : ['mean']
    }

    grouped_main_features = installments_payments_agg_prev.groupby('SK_ID_CURR').agg(main_features_aggregations)
    grouped_main_features.columns = ['_'.join(ele).upper() for ele in grouped_main_features.columns]

    #group remaining ones
    remaining_columns = [col for col in installments_payments_agg_prev.columns if col not in grouped_main_features.columns and col != 'SK_ID_CURR']
    grouped_remaining_features = installments_payments_agg_prev[['SK_ID_CURR'] + remaining_columns].groupby('SK_ID_CURR').mean()

    df_agg = grouped_main_features.merge(grouped_remaining_features, on = 'SK_ID_CURR', how = 'inner')
    df_agg = df_agg.reset_index()

    return df_agg

### 3. Main Function

In [152]:


def installments_payments():
    # Load data
    instal = pd.read_csv('data/installments_payments.csv')
    target = pd.read_csv('data/target.csv')

    # Set target index
    target.set_index('SK_ID_CURR', inplace=True)

    # Merge with target and split train/test
    instal_agg = instal.merge(target, on='SK_ID_CURR', how='left')
    instal_train = instal_agg[~instal_agg['TARGET'].isnull()]
    instal_test = instal_agg[instal_agg['TARGET'].isnull()]

    # clean
    # instal_train, instal_test = missing_values(instal_train, instal_test)
    instal_train, instal_test = missing_values_ins(instal_train, instal_test)
    instal_train, instal_test = clipping_outliers_app(instal_train, instal_test)
    # Process
    instal_train_processed = feature_engineering_ins(instal_train)
    instal_train_processed = instal_train_processed.replace([np.inf, -np.inf], np.nan)

    instal_test_processed = feature_engineering_ins(instal_test)
    instal_test_processed = instal_test_processed.replace([np.inf, -np.inf], np.nan)

    # Combine train and test sets
    processed_installments_payments = pd.concat([instal_train_processed, instal_test_processed], axis=0)
    # train = processed_installments_payments[processed_installments_payments['TARGET'].notna()]
    # test = processed_installments_payments[processed_installments_payments['TARGET'].isna()]
    # train = select_feature(train)
    # processed_installments_payments = pd.concat([train, test[train.columns]], axis=0)
    print('"Installments Payments" final shape:', processed_installments_payments.shape)
    return processed_installments_payments

### TEMPORARY

In [139]:
def installments_payments_temp():
    ins = pd.read_csv(r'data/installments_payments.csv')

    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)

    # Group payments and get Payment difference
    ins = do_sum(ins, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')
    ins['PAYMENT_DIFFERENCE'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT_GROUPED']
    ins['PAYMENT_RATIO'] = ins['AMT_INSTALMENT'] / ins['AMT_PAYMENT_GROUPED']
    ins['PAID_OVER_AMOUNT'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']
    ins['PAID_OVER'] = (ins['PAID_OVER_AMOUNT'] > 0).astype(int)

    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']

    # Days past due and days before due (no negative values)
    ins['DPD_diff'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD_diff'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD_diff'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD_diff'].apply(lambda x: x if x > 0 else 0)

    # Flag late payment
    ins['LATE_PAYMENT'] = ins['DBD'].apply(lambda x: 1 if x > 0 else 0)
    ins['INSTALMENT_PAYMENT_RATIO'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['LATE_PAYMENT_RATIO'] = ins.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)

    # Flag late payments that have a significant amount
    ins['SIGNIFICANT_LATE_PAYMENT'] = ins['LATE_PAYMENT_RATIO'].apply(lambda x: 1 if x > 0.05 else 0)
    
    # Flag k threshold late payments
    ins['DPD_7'] = ins['DPD'].apply(lambda x: 1 if x >= 7 else 0)
    ins['DPD_15'] = ins['DPD'].apply(lambda x: 1 if x >= 15 else 0)

    ins['INS_IS_DPD_UNDER_120'] = ins['DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    ins['INS_IS_DPD_OVER_120'] = ins['DPD'].apply(lambda x: 1 if (x >= 120) else 0)

    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum', 'var'],
        'DBD': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum', 'min'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum', 'min'],
        'SK_ID_PREV': ['size', 'nunique'],
        'PAYMENT_DIFFERENCE': ['mean'],
        'PAYMENT_RATIO': ['mean', 'max'],
        'LATE_PAYMENT': ['mean', 'sum'],
        'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
        'LATE_PAYMENT_RATIO': ['mean'],
        'DPD_7': ['mean'],
        'DPD_15': ['mean'],
        'PAID_OVER': ['mean'],
        'DPD_diff':['mean', 'min', 'max'],
        'DBD_diff':['mean', 'min', 'max'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

    # from oof (DAYS_ENTRY_PAYMENT)
    cond_day = ins['DAYS_ENTRY_PAYMENT'] >= -365
    ins_d365_grp = ins[cond_day].groupby('SK_ID_CURR')
    ins_d365_agg_dict = {
        'SK_ID_CURR': ['count'],
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DAYS_ENTRY_PAYMENT': ['mean', 'max', 'sum'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_PAYMENT': ['mean', 'max', 'sum'],
        'PAYMENT_DIFF': ['mean', 'min', 'max', 'sum'],
        'PAYMENT_PERC': ['mean', 'max'],
        'DPD_diff': ['mean', 'min', 'max'],
        'DPD': ['mean', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']}

    ins_d365_agg = ins_d365_grp.agg(ins_d365_agg_dict)
    ins_d365_agg.columns = ['INS_D365' + ('_').join(column).upper() for column in ins_d365_agg.columns.ravel()]

    ins_agg = ins_agg.merge(ins_d365_agg, on='SK_ID_CURR', how='left')
    ins_agg.reset_index(inplace=True)
    print('"Installments Payments" final shape:', ins_agg.shape)
    return ins_agg

## V. Pos Cash Balance 

### 1.Clean Data

In [140]:
def one_hot_encoder_pos(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(include='object').columns.tolist()
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
def do_sum_pos(dataframe, group_cols, counted, agg_name):
    gp = dataframe[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(columns={counted: agg_name})
    dataframe = dataframe.merge(gp, on=group_cols, how='left')
    return dataframe

# convert days to years, remove invalid value, drop col with large number of missing value
def prep_fill_missing_pos(dataframe):
    # Replace specific invalid values with NaN
    values_to_replace = ['XNA', 'XAP']
    dataframe.replace(values_to_replace, np.nan, inplace=True)

    return dataframe

def lazy_fill_missing_pos(dataframe_train, dataframe_test):
    col_num = dataframe_train.select_dtypes('number').columns.drop('TARGET')
    for col in col_num:
        median_value = dataframe_train[col].median()
        dataframe_train[col] = dataframe_train[col].fillna(median_value)
        dataframe_test[col] = dataframe_test[col].fillna(median_value)
    col_object = dataframe_train.select_dtypes('object').columns
    for col in col_object:
        mode_value = dataframe_train[col].mode()[0]
        dataframe_train[col] = dataframe_train[col].fillna(mode_value)
        dataframe_test[col] = dataframe_test[col].fillna(mode_value)
    return dataframe_train, dataframe_test

# fill by mean (for numeric column) or by mode (for category column) by each specified column
def fill_missing_pos(dataframe_train, dataframe_test):

    # FILL BY 'AMT_APPLICATION'
    dataframe_train['MONTH_BALANCE_BINS'] = pd.cut(dataframe_train['MONTHS_BALANCE'], 10)
    dataframe_test['MONTH_BALANCE_BINS'] = pd.cut(dataframe_test['MONTHS_BALANCE'], 10)

    col_to_group = 'MONTH_BALANCE_BINS'
    col_to_impute = ['CNT_INSTALMENT_FUTURE', 'CNT_INSTALMENT']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        grouped = dataframe_train.groupby([col_to_group])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mode_value
    dataframe_train.drop(columns='MONTH_BALANCE_BINS', inplace=True)
    dataframe_test.drop(columns='MONTH_BALANCE_BINS', inplace=True)
    return dataframe_train, dataframe_test


### 2.Feature Engineering

In [141]:
# Banr moi.
def create_feature_pos(df):
    # Drop duplicates for cleaning
    df = df.drop_duplicates()

    # One-hot encoding of categorical variables
    df, cat_cols = one_hot_encoder_pos(df, nan_as_category=True)

    # Feature engineering for late payment behavior
    df['LATE_PAYMENT'] = df['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    df['POS_IS_DPD'] = df['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    df['POS_IS_DPD_UNDER_120'] = df['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    df['POS_IS_DPD_OVER_120'] = df['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # Weighted payment efficiency (based on time) cai nao gan ngay hon, thi weight nang hon
    df['DAYS_WEIGHT'] = 1 / (df['MONTHS_BALANCE'] + 1)  # Avoid division by zero
    df['WEIGHTED_PAYMENT_EFFICIENCY'] = df['SK_DPD'] * df['DAYS_WEIGHT']

    # Aggregations for key metrics
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size', 'min'],
        'SK_DPD': ['max', 'mean', 'sum', 'var', 'min'],
        'SK_DPD_DEF': ['max', 'mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'LATE_PAYMENT': ['mean'],
        'CNT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'sum'],
        'POS_IS_DPD': ['mean', 'sum'],
        'POS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'POS_IS_DPD_OVER_120': ['mean', 'sum'],
        'WEIGHTED_PAYMENT_EFFICIENCY': ['mean', 'sum']
    }

    # Include categorical aggregations
    for cat in cat_cols:
        aggregations[cat] = ['mean']

    # Perform aggregation
    df_agg = df.groupby('SK_ID_CURR').agg(aggregations)
    df_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
    df_agg['POS_COUNT'] = df.groupby('SK_ID_CURR').size()

    # # Add trend features from the most recent applications
    # last_month_df = df.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    # last_month_records = df.loc[last_month_df]
    # recent_gp = (
    #     last_month_records
    #     .sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE'], ascending=[True, False])
    #     .groupby('SK_ID_CURR')
    #     .head(3)
    # )

    # recent_gp_agg = recent_gp.groupby('SK_ID_CURR').mean().reset_index()
    # df_agg = pd.merge(df_agg, recent_gp_agg[['SK_ID_CURR', 'LATE_PAYMENT']], on='SK_ID_CURR', how='left', suffixes=('', '_LAST3'))

    # # Add percentage of late payments for last 3 applications
    # df_agg['POS_LATE_PAYMENT_RATIO_LAST3'] = df_agg['LATE_PAYMENT_LAST3'] / 3
        # Find the most recent 3 records for each SK_ID_CURR
    last_month_df = df.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()  # Most recent month per previous ID
    last_month_records = df.loc[last_month_df]  # Subset of most recent records
    gp = (
        last_month_records
        .sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE'], ascending=[True, False])  # Sort by recency
        .groupby('SK_ID_CURR')
        .head(3)  # Top 3 most recent applications
    )

    # Calculate the mean of LATE_PAYMENT for these 3 applications
    late_payment_last3 = gp.groupby('SK_ID_CURR')['LATE_PAYMENT'].mean().reset_index()
    late_payment_last3.rename(columns={'LATE_PAYMENT': 'LATE_PAYMENT_LAST3'}, inplace=True)

    # Merge this feature back into the aggregated DataFrame
    df_agg = pd.merge(df_agg, late_payment_last3, on='SK_ID_CURR', how='left')

    print('"Pos-Cash" balance final shape:', df_agg.shape)
    return df_agg


### 3.Main Function

In [142]:
def pos_cash():
    """Process the POS_CASH_balance data by merging, transforming, and scaling."""
    # Load data
    pos = pd.read_csv('data/POS_CASH_balance.csv')
    app_train = pd.read_csv('data/application_train.csv')
    app_test = pd.read_csv('data/application_test.csv')
    app_full = pd.concat([app_train, app_test], axis=0)
    target = app_full[['SK_ID_CURR','TARGET']]
    target.set_index('SK_ID_CURR', inplace=True)
    pos = pos.merge(target, on='SK_ID_CURR', how='inner')
    # split train and test
    train = pos[~pos['TARGET'].isnull()]
    test = pos[pos['TARGET'].isnull()]

    #prepare before filling missing
    train = prep_fill_missing_pos(train)
    test = prep_fill_missing_pos(test)
     # fill missing
    train, test = fill_missing_pos(train, test)
    # train, test = lazy_fill_missing_pos(train, test)
    train, test = clipping_outliers_app(train, test)
    #FE
    train = create_feature_pos(train)
    test = create_feature_pos(test)
    train = train.merge(target, on='SK_ID_CURR', how='left')
    train.replace([np.inf, -np.inf], np.nan, inplace=True)
    test = test.merge(target, on='SK_ID_CURR', how='left')
    test.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Combine train and test back after cleaning
    train = select_feature(train)
    df = pd.concat([train, test[train.columns]], axis=0)
    df.drop(columns='TARGET', inplace=True)
    print('"Application_Train_Test" final shape:', df.shape)
    return df

### TEMPORARY

In [143]:
def pos_cash_temp():
    pos = pd.read_csv(r'data/POS_CASH_balance.csv')

    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)

    # Flag months with late payment
    pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    pos['POS_IS_DPD'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0) # <-- same with ['LATE_PAYMENT']
    pos['POS_IS_DPD_UNDER_120'] = pos['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    pos['POS_IS_DPD_OVER_120'] = pos['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)

    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size', 'min'],
        'SK_DPD': ['max', 'mean', 'sum', 'var', 'min'],
        'SK_DPD_DEF': ['max', 'mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'LATE_PAYMENT': ['mean'],
        'SK_ID_CURR': ['count'],
        'CNT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'sum'],
        'POS_IS_DPD': ['mean', 'sum'],
        'POS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'POS_IS_DPD_OVER_120': ['mean', 'sum'],
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']

    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])

    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()


    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.groupby('SK_ID_PREV')
    df_pos = pd.DataFrame()
    df_pos['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
    df_pos['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()

    # Percentage of previous loans completed and completed before initial term
    df_pos['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
    df_pos['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - gp['CNT_INSTALMENT'].last()
    df_pos['POS_COMPLETED_BEFORE_MEAN'] = df_pos.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0 \
                                                                      and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis=1)
    # Number of remaining installments (future installments) and percentage from total
    df_pos['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
    df_pos['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last()/gp['CNT_INSTALMENT'].last()

    # Group by SK_ID_CURR and merge
    df_gp = df_pos.groupby('SK_ID_CURR').sum().reset_index()
    df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace= True)
    pos_agg = pd.merge(pos_agg, df_gp, on= 'SK_ID_CURR', how= 'left')

    # Percentage of late payments for the 3 most recent applications
    pos = do_sum(pos, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')

    # Last month of each application
    last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()

    # Most recent applications (last 3)
    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)
    gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
    pos_agg = pd.merge(pos_agg, gp_mean[['SK_ID_CURR', 'LATE_PAYMENT_SUM']], on='SK_ID_CURR', how='left')

    print('"Pos-Cash" balance final shape:', pos_agg.shape) 
    return pos_agg

In [144]:
# pos = pos_cash()
# pos.head()

## VI. Previous Application 

### 1.Clean Data

In [145]:
#help function
def one_hot_encoder_prev_app(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(include='object').columns.tolist()
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


# convert days to years, remove invalid value, drop col with large number of missing value
def prep_fill_missing_prev_app(dataframe):

    # convert days to years
    def days_to_year(dataframe):
        for col in dataframe.columns[dataframe.columns.str.startswith('DAYS')]:
            dataframe[f'YEARS{col[4:]}'] = np.ceil(dataframe[col] / -365)

        return dataframe
    dataframe = days_to_year(dataframe)

    # delete invalid in years (< 0)
    dataframe[['YEARS_DECISION', 'YEARS_FIRST_DRAWING', 'YEARS_FIRST_DUE', 'YEARS_LAST_DUE_1ST_VERSION', 'YEARS_LAST_DUE', 'YEARS_TERMINATION']].applymap(lambda x: np.nan if x < 0 else x)
    # delete invalid in days (> 0)
    dataframe[['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']] = dataframe[['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']].applymap(lambda x: np.nan if x > 0 else x)

    # drop >95% missing
    col_to_drop = ['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED']
    dataframe.drop(columns=col_to_drop, inplace=True)

    # Replace specific invalid DAYS values with NaN
    cols_to_replace_nan = ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
                           'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']
    for col in cols_to_replace_nan:
        if col in dataframe.columns:
            dataframe[col].replace(365243, np.nan, inplace=True)

    # Replace specific invalid values with NaN
    values_to_replace = ['XNA', 'XAP']
    dataframe.replace(values_to_replace, np.nan, inplace=True)

    return dataframe

def lazy_fill_missing_prev(dataframe_train, dataframe_test):
    col_num = dataframe_train.select_dtypes('number').columns.drop('TARGET')
    for col in col_num:
        median_value = dataframe_train[col].median()
        dataframe_train[col] = dataframe_train[col].fillna(median_value)
        dataframe_test[col] = dataframe_test[col].fillna(median_value)
    col_object = dataframe_train.select_dtypes('object').columns
    for col in col_object:
        mode_value = dataframe_train[col].mode()[0]
        dataframe_train[col] = dataframe_train[col].fillna(mode_value)
        dataframe_test[col] = dataframe_test[col].fillna(mode_value)
    return dataframe_train, dataframe_test

# fill by mean (for numeric column) or by mode (for category column) by each specified column
def fill_missing_prev_app(dataframe_train, dataframe_test):
    # NAME_TYPE_SUITE: ng di cung khi vay fill = mode
    # mode_suite = dataframe_train['NAME_TYPE_SUITE'].mode()
    # dataframe_train['NAME_TYPE_SUITE'].fillna(mode_suite, inplace=True)
    # dataframe_test['NAME_TYPE_SUITE'].fillna(mode_suite, inplace=True)

    # PRODUCT_COMBINATION
    mode_combination = dataframe_train['PRODUCT_COMBINATION'].mode()
    dataframe_train['PRODUCT_COMBINATION'].fillna(mode_combination, inplace=True)
    dataframe_test['PRODUCT_COMBINATION'].fillna(mode_combination, inplace=True)

    # FILL BY 'AMT_APPLICATION'
    dataframe_train['AMT_APP_BINS'] = pd.cut(dataframe_train['AMT_APPLICATION'], 10)
    dataframe_test['AMT_APP_BINS'] = pd.cut(dataframe_test['AMT_APPLICATION'], 10)

    col_to_group = 'AMT_APP_BINS'
    col_to_impute = ['AMT_GOODS_PRICE', 'AMT_ANNUITY','AMT_DOWN_PAYMENT','RATE_DOWN_PAYMENT', 'AMT_CREDIT','NAME_TYPE_SUITE']
    # vòng for dùng để impute cho nhiều cột
    for i, col in enumerate(col_to_impute):
        # sau khi group kiểu này, grouped chứa 2 phần tử: val_to_group (giá trị được group, ở đây là từng Age_bin) và group_data (dataframe có Age_bin = val_to_group)
        grouped = dataframe_train.groupby([col_to_group])[col]
        for val_to_group, group_data in grouped:
            if dataframe_train[col].dtypes in [int, float]:  # fill biến numeric = mean của từng group_data
                mean_value = group_data.mean()
                # phải dùng val_to_group[0] là bởi val_to_group được trả về là tuple do hàm cut ban đầu (đừng quan tâm cái này vội)
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mean_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mean_value
            else:  # fill biến câte = mode của từng group_data
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                dataframe_train.loc[(dataframe_train[col_to_group] == val_to_group[0]) & (dataframe_train[col].isna()), col] = mode_value
                dataframe_test.loc[(dataframe_test[col_to_group] == val_to_group[0]) & (dataframe_test[col].isna()), col] = mode_value
    dataframe_train.drop(columns='AMT_APP_BINS', inplace=True)
    dataframe_test.drop(columns='AMT_APP_BINS', inplace=True)
    return dataframe_train, dataframe_test


### 2.Feature Engineering

In [146]:
def create_feature_prev_app(df):
    # Preserve the original PRODUCT_COMBINATION column
    product_combination = df[['SK_ID_CURR', 'DAYS_DECISION', 'PRODUCT_COMBINATION']].copy()

    # Apply one-hot encoding
    df, cat_cols = one_hot_encoder_prev_app(df, nan_as_category=True)

    # Restore the PRODUCT_COMBINATION column
    df = df.merge(product_combination, on=['SK_ID_CURR', 'DAYS_DECISION'], how='left')

    # Ratios and Differences
    df['APP_CREDIT_PERC'] = df['AMT_APPLICATION'] / df['AMT_CREDIT']
    df['APPLICATION_CREDIT_DIFF'] = df['AMT_APPLICATION'] - df['AMT_CREDIT']
    df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['DOWN_PAYMENT_TO_CREDIT'] = df['AMT_DOWN_PAYMENT'] / df['AMT_CREDIT']
    df['SIMPLE_INTERESTS'] = (df['AMT_ANNUITY'] * df['CNT_PAYMENT'] / df['AMT_CREDIT'] - 1) / df['CNT_PAYMENT']
    df['DAYS_LAST_DUE_DIFF'] = df['DAYS_LAST_DUE_1ST_VERSION'] - df['DAYS_LAST_DUE']
    df['PREV_GOODS_DIFF'] = df['AMT_APPLICATION'] - df['AMT_GOODS_PRICE']
    df['PREV_ANNUITY_APPL_RATIO'] = df['AMT_ANNUITY'] / df['AMT_APPLICATION']
    df['PREV_GOODS_APPL_RATIO'] = df['AMT_GOODS_PRICE'] / df['AMT_APPLICATION']

    # Add prev_PRODUCT_COMBINATION: Most recent PRODUCT_COMBINATION
    df['RECENT_PRODUCT_COMBINATION'] = df.sort_values('DAYS_DECISION').groupby('SK_ID_CURR')['PRODUCT_COMBINATION'].transform('last')

    # Aggregations for numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean', 'sum'],
        'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'DAYS_TERMINATION': ['max'],
        'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
        'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'sum'],
        'DOWN_PAYMENT_TO_CREDIT': ['mean'],
        'PREV_GOODS_DIFF': ['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO': ['mean', 'max'],
        'DAYS_LAST_DUE_DIFF': ['mean', 'max', 'sum'],
        'SIMPLE_INTERESTS': ['mean', 'max']
    }

    # Categorical feature aggregation
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    # Aggregating for all previous applications
    df_agg = df.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    df_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])

    # Add PRODUCT_COMBINATION
    recent_combination = df.groupby('SK_ID_CURR')['RECENT_PRODUCT_COMBINATION'].last()
    df_agg['PREV_RECENT_PRODUCT_COMBINATION'] = recent_combination

    # Aggregate Last 3, 5 and First 2, 4 Applications
    def aggregate_subsets(df, n_first, n_last):
        first_n = df.groupby('SK_ID_CURR').head(n_first).groupby('SK_ID_CURR').agg(num_aggregations)
        first_n.columns = [f'FIRST{n_first}_' + e[0] + "_" + e[1].upper() for e in first_n.columns]

        last_n = df.groupby('SK_ID_CURR').tail(n_last).groupby('SK_ID_CURR').agg(num_aggregations)
        last_n.columns = [f'LAST{n_last}_' + e[0] + "_" + e[1].upper() for e in last_n.columns]

        return first_n, last_n

    for n_first, n_last in [(2, 3), (4, 5)]:
        first_n, last_n = aggregate_subsets(df, n_first, n_last)
        df_agg = df_agg.join(first_n, how='left', on='SK_ID_CURR')
        df_agg = df_agg.join(last_n, how='left', on='SK_ID_CURR')

    # Approved and Refused Applications Aggregation
    for status, prefix in zip(['Approved', 'Refused'], ['APPROVED', 'REFUSED']):
        subset = df[df[f'NAME_CONTRACT_STATUS_{status}'] == 1]
        subset_agg = subset.groupby('SK_ID_CURR').agg(num_aggregations)
        subset_agg.columns = pd.Index([f'{prefix}_' + e[0] + "_" + e[1].upper() for e in subset_agg.columns.tolist()])
        df_agg = df_agg.join(subset_agg, how='left', on='SK_ID_CURR')
    
    return df_agg

### 3.Main Function

In [156]:
def previous_application():
  data = pd.read_csv('data/previous_application.csv')
  app_train = pd.read_csv('data/application_train.csv')
  app_test = pd.read_csv('data/application_test.csv')
  app_full = pd.concat([app_train, app_test], axis=0)
  target = app_full[['SK_ID_CURR','TARGET']]
  target.set_index('SK_ID_CURR', inplace=True)
  data = data.merge(target, on='SK_ID_CURR', how='inner')

  # split train and test
  train = data[~data['TARGET'].isnull()]
  test = data[data['TARGET'].isnull()]

  #prepare before filling missing
  train = prep_fill_missing_prev_app(train)
  test = prep_fill_missing_prev_app(test)

  # fill missing
  # train, test = fill_missing_prev_app(train, test)
  train, test = fill_missing_prev_app(train, test)
  train, test = clipping_outliers_app(train, test)
  #FE
  train = create_feature_prev_app(train)
  test = create_feature_prev_app(test)
  train = train.merge(target, on='SK_ID_CURR', how='left').reset_index()
  train.replace([np.inf, -np.inf], np.nan, inplace=True)
  test = test.merge(target, on='SK_ID_CURR', how='left').reset_index()
  test.replace([np.inf, -np.inf], np.nan, inplace=True)

  # Combine train and test back after cleaning
  train = select_feature(train)
  df = pd.concat([train, test[train.columns]], axis=0)
  df.drop(columns='TARGET', inplace=True)

  df, cat_cols = one_hot_encoder_prev_app(df, nan_as_category=True)

  print('Previous Application final shape:', df.shape)
  return df

### TEMPORARY

In [148]:
def previous_application_temp():
    prev = pd.read_csv('data/previous_application.csv')

    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)

    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

    # Feature engineering: ratios and difference
    prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT'] / prev['AMT_ANNUITY']
    prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']

    # Interest ratio on previous application (simplified)
    total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    prev['SIMPLE_INTERESTS'] = (total_payment / prev['AMT_CREDIT'] - 1) / prev['CNT_PAYMENT']

    # Days last due difference (scheduled x done)
    prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']

    # from off
    prev['PREV_GOODS_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_GOODS_PRICE']
    prev['PREV_ANNUITY_APPL_RATIO'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION']
    prev['PREV_GOODS_APPL_RATIO'] = prev['AMT_GOODS_PRICE'] / prev['AMT_APPLICATION']

    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean', 'sum'],
        'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'SK_ID_PREV': ['nunique'],
        'DAYS_TERMINATION': ['max'],
        'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
        'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'sum'],
        'DOWN_PAYMENT_TO_CREDIT': ['mean'],
        'PREV_GOODS_DIFF': ['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO': ['mean', 'max'],
        'DAYS_LAST_DUE_DIFF': ['mean', 'max', 'sum'],
        'SIMPLE_INTERESTS': ['mean', 'max']
    }

    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    prev_agg.reset_index(inplace=True)
    print('"Previous Applications" final shape:', prev_agg.shape)
    return prev_agg

## Modeling

In [ ]:
class DataProcessor:
    def __init__(self, train, test, target, *dfs):
        self.train = train
        self.test = test
        self.target = target
        self.dfs = dfs
        
        self.features = None
        self.imputer = SimpleImputer(strategy='mean').set_output(transform='pandas')
        self.scaler = StandardScaler().set_output(transform='pandas')

    def process(self, train, test, target):
        # Set index and sort dataframes
        train.set_index('SK_ID_CURR', inplace=True)
        train.sort_index(inplace=True)
        test.set_index('SK_ID_CURR', inplace=True)
        test.sort_index(inplace=True)
        target.set_index('SK_ID_CURR', inplace=True)
        target.sort_index(inplace=True)
        target = target['TARGET']

        # Add is_train column and merge train and test data
        train['is_train'] = 1
        test['is_train'] = 0
        data = pd.concat([train, test], axis=0)

        # Merge additional dataframes 
        for df in self.dfs:
            data = data.merge(df, how='left', on='SK_ID_CURR')
        # data['annuity_to_max_installment_ratio'] = data['AMT_ANNUITY'] / data['INSTAL_AMT_PAYMENT_MAX']
        # Remove duplicated columns, replace infinite values with NaN, and drop TARGET column
        data = data.loc[:, ~data.columns.duplicated()]
        data.replace([np.inf, -np.inf], np.nan, inplace=True)
        data.drop(['TARGET'], axis=1, inplace=True, errors='ignore')
        data.set_index('SK_ID_CURR', inplace=True)
        print(f'Merged data shape: {data.shape}')
        # Split train and test data
        train = data[data['is_train'] == 1].drop(['is_train'], axis=1)
        test = data[data['is_train'] == 0].drop(['is_train'], axis=1)

        # # Sanitize columns and convert data types to float64
        # train = sanitize_columns(train)
        # test = sanitize_columns(test)
        # print(train.shape)
        train = train.astype('float64')
        test = test.astype('float64')
        return train, test, target

    def transform(self, data):
        data = self.imputer.transform(data)
        data = self.scaler.transform(data)
        return data
    
    def fit_transform(self, train, target):
        train = self.imputer.fit_transform(train)
        train = self.scaler.fit_transform(train)
        return train


In [ ]:
start = time.time()
app_train_test = application()
end = time.time()
print(f'Time for application : {np.round(end - start, 2)}s')


# # disable since second run
start = time.time()
# bureau_temp = bureau_and_balance_temp() 
bureau = bureau_and_balance()
end = time.time()
print(f'Time for bureau_bb : {np.round(end - start, 2)}s')

start = time.time()
# credit_card_temp = credit_card_balance_temp() 
credit_card = credit_card_balance() 
end = time.time()
print(f'Time for credit card : {np.round(end - start, 2)}s')

start = time.time()
# ins_temp = installments_payments_temp() 
ins = installments_payments() 
end = time.time()
print(f'Time for installments : {np.round(end - start, 2)}s')

start = time.time()
# pos_cash_balance = pos_cash_temp() 
pos_cash_balance = pos_cash() 
end = time.time()
print(f'Time for posh cash : {np.round(end - start, 2)}s')

start = time.time()
# prev_app_temp = previous_application_temp() 
prev_app = previous_application() 
end = time.time()
print(f'Time for previous application : {np.round(end - start, 2)}s')




"Credit Card Balance" final shape: (85322, 168)
Time for credit card : 70.86s
"Installments Payments" final shape: (180733, 211)
Time for installments : 27.37s
"Pos-Cash" balance final shape: (231531, 37)
"Pos-Cash" balance final shape: (57913, 37)
keep: 11 
drop iv: 20 
drop empty: 0 
drop corr: 7
"Application_Train_Test" final shape: (289444, 10)
Time for posh cash : 35.95s
keep: 93 
drop iv: 220 
drop empty: 54 
drop corr: 97
Previous Application final shape: (291057, 101)
Time for previous application : 134.75s


In [176]:

app_train = app_train_test[app_train_test['TARGET'].notna()]
app_test= app_train_test[app_train_test['TARGET'].isna()]
target = app_train_test[['SK_ID_CURR', 'TARGET']].dropna()

print(f'app_train : {app_train.shape}')
print(f'app_test : {app_test.shape}')
print(f'target : {target.shape}')

print(f'previous_application : {prev_app.shape}')
print(f'credit_card_balance : {credit_card.shape}')
print(f'installments_payments : {ins.shape}')
print(f'bureau : {bureau.shape}')
print(f'pos_cash_balance : {pos_cash_balance.shape}')

app_train : (246009, 59)
app_test : (61502, 59)
target : (246009, 2)
previous_application : (291057, 101)
credit_card_balance : (85322, 168)
installments_payments : (180733, 211)
bureau : (263491, 52)
pos_cash_balance : (289444, 10)


In [177]:
start = time.time()
# Initialize DataProcessor object
processor = DataProcessor(app_train, app_test, target,
                          prev_app,credit_card, 
                          ins, bureau, pos_cash_balance)

# Process train and test data, fit and transform train data, and transform test data
train, test, target = processor.process(app_train, app_test, target)
train = processor.fit_transform(train, target)
test = processor.transform(test)
end = time.time()
print(f"Processing time: {end - start:.2f}s")
print(f'train : {train.shape}')
print(f'target : {target.shape}')
print(f'test : {test.shape}')

Merged data shape: (307511, 595)
Processing time: 23.90s
train : (246009, 565)
target : (246009,)
test : (61502, 565)


In [178]:

model_logis = LogisticRegression(penalty='l2', class_weight='balanced', C=0.001, solver='newton-cholesky', max_iter=1000)
# Cross-validation predictions
print('Cross validating and calculating metrics...')
y_pred_proba = cross_val_predict(model_logis, train, target, cv=5, method='predict_proba')[:, 1]  # Predicted probabilities
y_pred = cross_val_predict(model_logis, train, target, cv=5, method='predict')  # Predicted classes

# Calculate metrics
roc_auc = roc_auc_score(target, y_pred_proba)
precision = precision_score(target, y_pred)
recall = recall_score(target, y_pred)
f1 = f1_score(target, y_pred)

# Print results
print(f'ROC AUC: {roc_auc}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
print('\nClassification Report:')
print(classification_report(target, y_pred))

Cross validating and calculating metrics...
ROC AUC: 0.7690999369017111
Precision: 0.1729006440317639
Recall: 0.6956127993560073
F1-score: 0.2769603669835038

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.71      0.82    226133
         1.0       0.17      0.70      0.28     19876

    accuracy                           0.71    246009
   macro avg       0.57      0.70      0.55    246009
weighted avg       0.90      0.71      0.77    246009



In [175]:
# model = model_logis
# y_pred = model.predict_proba(test)[:, 1]
# submission = pd.DataFrame(index=test.index, data={'TARGET': y_pred})
# submission.sort_index(inplace=True)

# submission.to_csv(f'submit/submit_34_final_{np.round(gini_score, 4)}.csv')  